# Decode stimuli/block/licks from neural activity - DR

In [ ]:
#Import packages
import pandas as pd
import h5py
import numpy as np
import scipy as sp
import scipy.signal as sg
import scipy.stats as st
import xarray as xr
import os
import matplotlib.pyplot as plt
from matplotlib import patches
import ast
from sklearn import svm
import glob
import re
import pickle

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold

from DR_analysis_utils import Session, makePSTH, make_neuron_time_trials_tensor, compute_smoothed_response_rate

%matplotlib notebook

In [ ]:
#Set paths to experiment folders
main_path = [
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_626791_20220815\processed",
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_626791_20220816\processed",
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_626791_20220817\processed",
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_636766_20230123\processed", 
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_636766_20230124\processed", 
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_636766_20230125\processed", 
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_636766_20230126\processed", 
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_644864_20230130\processed",
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_644864_20230131\processed", 
    r"\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_644864_20230201\processed", 
    r"\\allen\programs\mindscope\workgroups\np-exp\PilotEphys\Task 2 pilot\DRpilot_644864_20230202\processed",
#     r"\\allen\programs\mindscope\workgroups\np-exp\PilotEphys\Task 2 pilot\DRpilot_644866_20230207\processed", 
    r"Y:\DRpilot_644866_20230208\processed",
    r"Y:\DRpilot_644866_20230209\processed",
    r"Y:\DRpilot_644866_20230210\processed",
    r"Y:\DRpilot_644867_20230220\processed",
    r"Y:\DRpilot_644867_20230221\processed",
    r"Y:\DRpilot_644867_20230222\processed",
    r"Y:\DRpilot_644867_20230223\processed",
    r"Y:\DRpilot_649943_20230213\processed", 
    r"Y:\DRpilot_649943_20230214\processed",
    r"Y:\DRpilot_649943_20230215\processed",
    r"Y:\DRpilot_649943_20230216\processed",
]



## loop through experiments, save all relevant info for each session

In [ ]:
def custom_decoder(input_data,labels):
    
    output={}
    
    scaler = StandardScaler()
    skf = StratifiedKFold(n_splits=5,shuffle=True)
    
    scaler.fit(sel_data.T)
    X = scaler.transform(input_data)
    y = labels
    
    if len(np.unique(labels))>2:
        y_dec_func=np.full((len(y),len(np.unique(labels))), fill_value=np.nan)
    else:
        y_dec_func=np.full(len(y), fill_value=np.nan)
 
    if type(y[0])==bool:
        ypred=np.full(len(y), fill_value=False)
    elif type(y[0])==str:
        ypred=np.full(len(y), fill_value='       ')
    else:
        ypred=np.full(len(y), fill_value=np.nan)

    tidx_used=[]
    
    coefs=[]
    classes=[]
#     feature_names=[]
    intercept=[]
    params=[]

    for train,test in skf.split(X, y):
        clf=svm.LinearSVC(max_iter=5000)
        clf.fit(X[train],y[train])
        ypred[test] = clf.predict(X[test])
        y_dec_func[test] = clf.decision_function(X[test])
        tidx_used.append([test])
        coefs.append(clf.coef_)
        classes.append(clf.classes_)
#         feature_names.append(clf.feature_names_in_)
        intercept.append(clf.intercept_)
        params.append(clf.get_params())

    cr_dict=classification_report(y, ypred, output_dict=True)
    cr_df=pd.DataFrame.from_dict(cr_dict)

    output['cr']=cr_df
    output['pred_label']=ypred
    output['true_label']=y
    output['trial_sel_idx']=trial_sel
    output['trials_used']=tidx_used
    output['decision_function']=y_dec_func
    output['coefs']=coefs
    output['classes']=classes
#     output['feature_names']=feature_names
    output['intercept']=intercept
    output['params']=params
    
    return output


In [ ]:
def rand_forest_decoder(input_data,labels):
    ### make this and try it!!
#     output={}
    
#     scaler = StandardScaler()
#     skf = StratifiedKFold(n_splits=5,shuffle=True)
    
#     scaler.fit(sel_data.T)
#     X = scaler.transform(input_data)
#     y = labels
    
#     if len(np.unique(labels))>2:
#         y_dec_func=np.full((len(y),len(np.unique(labels))), fill_value=np.nan)
#     else:
#         y_dec_func=np.full(len(y), fill_value=np.nan)
 
#     if type(y[0])==bool:
#         ypred=np.full(len(y), fill_value=False)
#     elif type(y[0])==str:
#         ypred=np.full(len(y), fill_value='       ')
#     else:
#         ypred=np.full(len(y), fill_value=np.nan)

#     tidx_used=[]

#     for train,test in skf.split(X, y):
#         clf=svm.LinearSVC(max_iter=5000)
#         clf.fit(X[train],y[train])
#         ypred[test] = clf.predict(X[test])
#         y_dec_func[test] = clf.decision_function(X[test])
#         tidx_used.append([test])

#     cr_dict=classification_report(y, ypred, output_dict=True)
#     cr_df=pd.DataFrame.from_dict(cr_dict)

#     output['cr']=cr_df
#     output['pred_label']=ypred
#     output['true_label']=y
#     output['trial_sel_idx']=trial_sel
#     output['trials_used']=tidx_used
#     output['decision_function']=y_dec_func
    
#     return output

In [ ]:
def match_run_speed(session):

    binsize=1
    speed_bins=np.arange(0,np.ceil(session.trials['avg_run_speed'].max()),binsize)

    visblock_run, edges = np.histogram(session.trials.query('trialstimRewarded=="vis1"')['avg_run_speed'],bins=speed_bins)
    audblock_run, edges = np.histogram(session.trials.query('trialstimRewarded=="sound1"')['avg_run_speed'],bins=speed_bins)

    choose_trials=[]

    for ib,bb in enumerate(edges[:-1]):

        n_chosen_trials = np.min([visblock_run[ib],audblock_run[ib]])

        visblock_trials = session.trials.query('trialstimRewarded=="vis1" and \
                                                avg_run_speed>=@speed_bins[@ib] and \
                                                avg_run_speed<@speed_bins[@ib+1]').index

        audblock_trials = session.trials.query('trialstimRewarded=="sound1" and \
                                                avg_run_speed>=@speed_bins[@ib] and \
                                                avg_run_speed<@speed_bins[@ib+1]').index

        choose_trials.append(np.random.choice(visblock_trials,n_chosen_trials,replace=False))
        choose_trials.append(np.random.choice(audblock_trials,n_chosen_trials,replace=False))


    choose_trials = np.sort(np.hstack(choose_trials))
    
    return choose_trials


In [ ]:
trnum='all'
u_num='all'
u_min=20
n_repeats=1
binsize=0.2
time_bins=np.arange(-4,3,binsize)
balance_labels=0


# svc_results={}

for sel_session, path in enumerate(main_path[:]):
    
    svc_results={}
    
    session=[]
    session=Session(path=path) 
    session.assign_unit_areas()
    session=compute_smoothed_response_rate(session)
    
    # loop through sessions and make unit xarrays
    time_before_flash = 4
    trial_duration = 7
    bin_size = 0.02

    # Make tensor (3-D matrix [units,time,trials])
    trial_tensor = make_neuron_time_trials_tensor(session.good_units, session.spike_times, 
                                                  session.trials,time_before_flash, trial_duration, 
                                                  bin_size)
    # make xarray
    session.trial_da = xr.DataArray(trial_tensor, dims=("unit_id", "time", "trials"), 
                               coords={
                                   "unit_id": session.good_units.index.values,
                                   "time": np.arange(0, trial_duration, bin_size)-time_before_flash,
                                   "trials": session.trials.index.values
                                   })
    
    #shorten the area names to better lump together units
    #get rid of layers and/or sub-areas with dashes
    area_short = []
    for area in session.good_units['area']:
        if area=='N/A':
            short='N/A'
        elif area[:2]=='CA':
            short=area
        else:
            dig_ind=re.search(r"\d", area)
            dash_ind=re.search(r"-", area)
            if dig_ind!=None:
                short=area[:dig_ind.start()]
            elif dash_ind!=None:
                short=area[:dash_ind.start()]
            else:
                short=area
        area_short.append(short)
    session.good_units['area_short']=area_short
    area_counts=session.good_units['area_short'].value_counts()
    
#     predict=['stim_ids','block_ids','trial_response']
    predict=['block_ids']

    #save metadata about this session & decoder params
    svc_results['metadata']=session.metadata
    svc_results['trial_numbers']=trnum
    svc_results['unit_numbers']=u_num
    svc_results['min_n_units']=u_min
    svc_results['n_repeats']=n_repeats
    svc_results['time_bins']=time_bins
    svc_results['balance_labels']=balance_labels
    
    #loop through different labels to predict
    for p in predict:
        svc_results[p]={}
    
        #choose what variable to predict
        if p=='stim_ids':
            #exclude any trials that had opto stimulation
            if 'trialOptoVoltage' in session.trials.columns:
                trial_sel = session.trials.query('trialOptoVoltage.isnull() and trialStimID != "catch"').index
            else:
                trial_sel = session.trials.query('trialStimID != "catch"').index
                
            # grab the stimulus ids
            pred_var = session.trials['trialStimID'][trial_sel].values
    
        elif p=='block_ids':
            #exclude any trials that had opto stimulation
            if 'trialOptoVoltage' in session.trials.columns:
                trial_sel = session.trials.query('trialOptoVoltage.isnull()').index
            else:
                trial_sel = session.trials.index
                
            # or, use block IDs
            pred_var = session.trials['trialstimRewarded'][trial_sel].values
    
        elif p=='trial_response':
            #exclude any trials that had opto stimulation
            if 'trialOptoVoltage' in session.trials.columns:
                trial_sel = session.trials.query('trialOptoVoltage.isnull()').index
            else:
                trial_sel = session.trials.index
                
            #or, use whether mouse responded
            pred_var = session.trials['trial_response'][trial_sel].values

#         area_sel = area_counts[area_counts>=u_min].index
#         area_sel = ['all']
        area_sel = ['all']+list(area_counts[area_counts>=u_min].index)
        
        #loop through areas
        for aa in area_sel:
            if aa=='all':
                unit_sel = session.good_units.index.values
            else:
                unit_sel = session.good_units.query('area_short==@aa').index.values
            svc_results[p][aa]={}
            svc_results[p][aa]['n_units']=len(unit_sel)
            
            #loop through time bins
            for tt,t_start in enumerate(time_bins[:-1]):
                svc_results[p][aa][tt]={}
                
                #loop through repeats
                for nn in range(0,n_repeats):
                    
                    if u_num=='all':
                        unit_subset = unit_sel #np.random.choice(unit_sel,len(unit_sel),replace=False)
                    else:
                        unit_subset = np.random.choice(unit_sel,u_num,replace=False)
                    
                    #option to balance number of labels for training
                    if balance_labels:
                        subset_ind=[]
                        conds = np.unique(pred_var)
                        cond_count=[]

                        if trnum=='all':
                            for cc in conds:
                                cond_count.append(np.sum(pred_var==cc))
                            use_trnum=np.min(cond_count)
                        else:
                            use_trnum = trnum

                        for cc in conds:
                            cond_inds=np.where(pred_var==cc)[0]
                            if len(cond_inds)<use_trnum:
                                use_trnum=len(cond_inds)
                            subset_ind.append(np.random.choice(cond_inds,use_trnum,replace=False))   
                        subset_ind=np.sort(np.hstack(subset_ind))

#                         trial_sel=trial_sel[subset_ind]
#                         pred_var=pred_var[subset_ind]
                    else:
                        subset_ind=np.arange(0,len(trial_sel))


                    sel_data = session.trial_da.sel(time=slice(t_start,time_bins[tt+1]),
                                                    trials=trial_sel[subset_ind],
                                                    unit_id=unit_subset).mean(dim='time').values

                    svc_results[p][aa][tt][nn]=custom_decoder(
                        input_data=sel_data.T,
                        labels=pred_var[subset_ind].flatten())

                    svc_results[p][aa][tt][nn]['shuffle']=custom_decoder(
                        input_data=sel_data.T,
                        labels=np.random.choice(pred_var[subset_ind],len(pred_var),replace=False).flatten())

                    svc_results[p][aa][tt][nn]['trial_sel_idx']=trial_sel
                    svc_results[p][aa][tt][nn]['unit_sel_idx']=unit_subset
                    

            print(aa+' done')
            
    print(path+' done')
    
            
    savepath=r'C:\Users\ethan.mcbride\OneDrive - Allen Institute\DR decoding results\by_session\pre_post_stim_activity_all_units'

    with open(os.path.join(savepath,session.metadata['mouseID']+'_'+str(session.metadata['ephys_session_num'])+
                           'decoder_results_200ms_incl_pre_post_all_units.pkl'), 'wb') as handle:
        pickle.dump(svc_results, handle, protocol=pickle.HIGHEST_PROTOCOL)        

In [ ]:
svc_results[p][aa][tt][nn]=custom_decoder(
                        input_data=sel_data.T,
                        labels=pred_var[subset_ind].flatten())

In [ ]:
['all']+list(area_counts[area_counts>=u_min].index)

In [ ]:
svc_results[p][aa][tt][nn]

In [ ]:
# fig,ax=plt.subplots()
# ax.axhline(0,color='k')
# ax.plot(svc_results['block_ids']['MOs'][0][nn]['decision_function'])
cond_inds.shape
use_trnum
np.random.choice(cond_inds,use_trnum,replace=False)

## balance for running

In [ ]:
# balance for running
trnum='all'
u_num='all'
u_min=20
n_repeats=20
binsize=0.2
time_bins=np.arange(-0.2,0.6,binsize)
balance_labels=0
balance_running=0

# svc_results={}

for sel_session, path in enumerate(main_path):
    
    svc_results={}
    
    session=[]
    session=Session(path=path) 
    session.assign_unit_areas()
    session=compute_smoothed_response_rate(session)
    
    # loop through sessions and make unit xarrays
    time_before_flash = 0.25
    trial_duration = 1.0
    bin_size = 0.001

    # Make tensor (3-D matrix [units,time,trials])
    trial_tensor = make_neuron_time_trials_tensor(session.good_units, session.spike_times, 
                                                  session.trials,time_before_flash, trial_duration, 
                                                  bin_size)
    # make xarray
    session.trial_da = xr.DataArray(trial_tensor, dims=("unit_id", "time", "trials"), 
                               coords={
                                   "unit_id": session.good_units.index.values,
                                   "time": np.arange(0, trial_duration, bin_size)-time_before_flash,
                                   "trials": session.trials.index.values
                                   })
    
    #shorten the area names to better lump together units
    #get rid of layers and/or sub-areas with dashes
    area_short = []
    for area in session.good_units['area']:
        if area=='N/A':
            short='N/A'
        elif area[:2]=='CA':
            short=area
        else:
            dig_ind=re.search(r"\d", area)
            dash_ind=re.search(r"-", area)
            if dig_ind!=None:
                short=area[:dig_ind.start()]
            elif dash_ind!=None:
                short=area[:dash_ind.start()]
            else:
                short=area
        area_short.append(short)
    session.good_units['area_short']=area_short
    area_counts=session.good_units['area_short'].value_counts()

    #exclude any trials that had opto stimulation
    if 'trialOptoVoltage' in session.trials.columns:
        trial_sel = session.trials.query('trialOptoVoltage.isnull()').index
    else:
        trial_sel = session.trials.index
            
    if balance_running:
        run_trial_sel = match_run_speed(session)
        
    predict=['stim_ids','block_ids','trial_response']

    # grab the stimulus ids
    stim_ids = session.trials['trialStimID'][trial_sel].values
    # or, use block IDs
    block_ids = session.trials['trialstimRewarded'][trial_sel].values
    #or, use whether mouse responded
    trial_response = session.trials['trial_response'][trial_sel].values
    
    #save metadata about this session & decoder params
    svc_results['metadata']=session.metadata
    svc_results['trial_numbers']=trnum
    svc_results['unit_numbers']=u_num
    svc_results['min_n_units']=u_min
    svc_results['n_repeats']=n_repeats
    svc_results['time_bins']=time_bins
    svc_results['balance_labels']=balance_labels
    
    #loop through different labels to predict
    for p in predict:
        svc_results[p]={}
    
        #choose what variable to predict
        if p=='stim_ids':
            pred_var = stim_ids
        elif p=='block_ids':
            pred_var = block_ids
        elif p=='trial_response':
            pred_var = trial_response
            
        if balance_running:
            if p=='stim_ids':
                run_pred_var = session.trials['trialStimID'][run_trial_sel].values
            elif p=='block_ids':
                run_pred_var = session.trials['trialstimRewarded'][run_trial_sel].values
            elif p=='trial_response':
                run_pred_var = session.trials['trial_response'][run_trial_sel].values

        area_sel = area_counts[area_counts>=u_min].index
        # area_sel = ['all']
        
        #loop through areas
        for aa in area_sel:
            if aa=='all':
                unit_sel = session.good_units.index.values
            else:
                unit_sel = session.good_units.query('area_short==@aa').index.values
            svc_results[p][aa]={}
            svc_results[p][aa]['n_units']=len(unit_sel)
            
            #loop through time bins
            for tt,t_start in enumerate(time_bins[:-1]):
                svc_results[p][aa][tt]={}
                
                #loop through repeats
                for nn in range(0,n_repeats):
                    
                    if u_num=='all':
                        unit_subset = np.random.choice(unit_sel,len(unit_sel),replace=False)
                    else:
                        unit_subset = np.random.choice(unit_sel,u_num,replace=False)
                    
#                     #option to balance number of labels for training
####### need to fix this! potnetially sub-selecting pred_var too much!
#                     if balance_labels:
#                         subset_ind=[]
#                         conds = np.unique(pred_var)
#                         cond_count=[]

#                         if trnum=='all':
#                             for cc in conds:
#                                 cond_count.append(np.sum(pred_var==cc))
#                             use_trnum=np.min(cond_count)
#                         else:
#                             use_trnum = trnum

#                         for cc in conds:
#                             cond_inds=np.where(pred_var==cc)[0]
#                             if len(cond_inds)<use_trnum:
#                                 use_trnum=len(cond_inds)
#                             subset_ind.append(np.random.choice(cond_inds,use_trnum,replace=False))   
#                         subset_ind=np.sort(np.hstack(subset_ind))

#                         trial_sel=trial_sel[subset_ind]
#                         pred_var=pred_var[subset_ind]


                    sel_data = session.trial_da.sel(time=slice(t_start,time_bins[tt+1]),
                                                    trials=trial_sel,
                                                    unit_id=unit_subset).mean(dim='time').values

                    svc_results[p][aa][tt][nn]=custom_decoder(
                        input_data=sel_data.T,
                        labels=pred_var.flatten())
                    
                    svc_results[p][aa][tt][nn]['trial_sel_idx']=trial_sel
                    
                    if balance_running:
                        
                        run_sel_data = session.trial_da.sel(time=slice(t_start,time_bins[tt+1]),
                                                    trials=run_trial_sel,
                                                    unit_id=unit_subset).mean(dim='time').values
                        
                        svc_results[p][aa][tt][nn]['balance_running']=custom_decoder(
                            input_data=run_sel_data.T,
                            labels=run_pred_var.flatten())
                        
                        svc_results[p][aa][tt][nn]['balance_running']['trial_sel_idx']=run_trial_sel

                        
            print(aa+' done')
            
    print(path+' done')
    
            
    savepath=r'C:\Users\ethan.mcbride\OneDrive - Allen Institute\DR decoding results\by_session\balance_running'

    with open(os.path.join(savepath,session.metadata['mouseID']+'_'+str(session.metadata['ephys_session_num'])+
                           'decoder_results_200ms_balance_running.pkl'), 'wb') as handle:
        pickle.dump(svc_results, handle, protocol=pickle.HIGHEST_PROTOCOL)        

In [ ]:
loadpath=r'C:\Users\ethan.mcbride\OneDrive - Allen Institute\DR decoding results\by_session\pre_post_stim_activity'

svc_results={}
decoder_results=os.listdir(loadpath)

for ii,ff in enumerate(decoder_results):
    with open(os.path.join(loadpath,ff),'rb') as handle:
        svc_results[ii]=pickle.load(handle)
    print(ff)

In [ ]:
# all unique areas in results
all_areas=[]
for sel_session in range(0,len(svc_results)):
    all_areas.append(list(svc_results[sel_session]['block_ids'].keys()))
    
all_areas=np.unique(np.hstack(all_areas))

In [ ]:
# all_areas

In [ ]:
# make into more plottable format

####change this to deal with multiple #s of trials and average over re-runs of the SVC

plot_results={}
plot_shuffle_results={}

timepoints=time_bins[1:]

label_list=['vis1','vis2','sound1','sound2','True','False']
# label_list=['vis1','sound1','True','False']

# predict=['stim_ids','block_ids','trial_response']
predict=['block_ids']

# areas=['VISp','AUD','LG','MG','MOs','MRN','CP','MOp']
areas=all_areas

for sel_session in range(0,len(svc_results)):
    plot_results[sel_session]={}
    plot_shuffle_results[sel_session]={}
    for aa in areas:
        plot_results[sel_session][aa]={}
        plot_shuffle_results[sel_session][aa]={}
        for p in predict:
            plot_results[sel_session][aa][p]={}
            plot_shuffle_results[sel_session][aa][p]={}
            for ll in label_list:
                plot_results[sel_session][aa][p][ll]=np.zeros((len(timepoints),n_repeats))
                plot_results[sel_session][aa][p][ll][:]=np.nan
                
                plot_shuffle_results[sel_session][aa][p][ll]=np.zeros((len(timepoints),n_repeats))
                plot_shuffle_results[sel_session][aa][p][ll][:]=np.nan

    
# diff table for each timepoint?
# row = session
# columns = ['modality','A_vis','A_aud','B_vis','B_aud','C_vis','C_aud','F_vis','F_aud',]

decoder_acc_session_mean = {}
decoder_acc_session_shuffle_mean = {}
for aa in areas:
    decoder_acc_session_mean[aa]={}
    decoder_acc_session_shuffle_mean[aa]={}
    for p in predict:
        decoder_acc_session_mean[aa][p]={}
        decoder_acc_session_shuffle_mean[aa][p]={}
        for ll in label_list:
            
            decoder_acc_session_mean[aa][p][ll]=np.zeros((len(timepoints),
                                                           len(main_path)))
            decoder_acc_session_mean[aa][p][ll][:]=np.nan
            
            decoder_acc_session_shuffle_mean[aa][p][ll]=np.zeros((len(timepoints),
                                                           len(main_path)))
            decoder_acc_session_shuffle_mean[aa][p][ll][:]=np.nan
        

for sel_session in range(0,len(svc_results)):
    for p in predict:
        for aa in areas: 
            if aa in svc_results[sel_session][p].keys():
                for tt,tp in enumerate(timepoints):
                    for nn in range(0,n_repeats):
                        for ll in label_list:
                            if len(svc_results[sel_session][p][aa])>0:
                                if len(svc_results[sel_session][p][aa][tt])>0:
                                    if ll in svc_results[sel_session][p][aa][tt][nn]['cr'].keys():
                                        temp_perf=np.nanmean(svc_results[sel_session][p]
                                                          [aa][tt][nn]['cr'][ll]
                                                          [['precision','recall']].values)
                                        plot_results[sel_session][aa][p][ll][tt,nn]=temp_perf
                                        
                                        temp_shuff_perf=np.nanmean(svc_results[sel_session][p]
                                                          [aa][tt][nn]['shuffle']['cr'][ll]
                                                          [['precision','recall']].values)
                                        plot_shuffle_results[sel_session][aa][p][ll][tt,nn]=temp_shuff_perf

                    for ll in label_list:
                        if len(svc_results[sel_session][p][aa])>0:                    
                            decoder_acc_mean = np.nanmean(plot_results[sel_session][aa][p][ll][tt])
                            decoder_acc_session_mean[aa][p][ll][tt,sel_session] = decoder_acc_mean
    
                            decoder_acc_shuffle_mean = np.nanmean(plot_shuffle_results[sel_session][aa][p][ll][tt])
                            decoder_acc_session_shuffle_mean[aa][p][ll][tt,sel_session] = decoder_acc_shuffle_mean
# plot_results[area][label/stimulus][timepoint_idx][ntrials_idx,nunits_idx,n_repeats]

In [ ]:
svc_results[sel_session][p][aa][tt][nn]['balance_running']['cr']

In [ ]:
svc_results[0]['block_ids']['MOs'][0][0]#['cr']['vis1']

In [ ]:
predict_var='block_ids'
sel_stim='vis1'

fig,ax=plt.subplots(1,1)

ax.axvline(0,color='k',linestyle='--',linewidth=1)
ax.axhline(0.25,color='k',linestyle='--',linewidth=1)

ax.plot(time_bins[1:],decoder_acc_session_mean['VISp'][predict_var][sel_stim],'k',linewidth=0.75,alpha=0.5)
# ax.plot(time_bins[1:],decoder_acc_session_mean['AUDp'][predict_var][sel_stim],'m',linewidth=0.75,alpha=0.5)
ax.plot(time_bins[1:],decoder_acc_session_mean['MOs'][predict_var][sel_stim],'b',linewidth=0.75,alpha=0.5)
# ax.plot(time_bins[1:],decoder_acc_session_mean['MRN'][predict_var][sel_stim],'r',linewidth=0.75,alpha=0.5)
# ax.plot(time_bins[1:],decoder_acc_session_mean['CP'][predict_var][sel_stim],'g',linewidth=0.75,alpha=0.5)


visp_line=ax.plot(time_bins[1:],np.nanmean(decoder_acc_session_mean['VISp'][predict_var][sel_stim],axis=1)
        ,'k.-',linewidth=2)

visp_line_balance_running=ax.plot(time_bins[1:],
                                  np.nanmean(decoder_acc_session_shuffle_mean['VISp'][predict_var][sel_stim],axis=1)
                                  ,'k^--',linewidth=1.5)

# ax.plot(time_bins[1:],np.nanmean(decoder_acc_session_mean['AUDp'][predict_var][sel_stim],axis=1)
#         ,'m.-',linewidth=2)
mos_line=ax.plot(time_bins[1:],np.nanmean(decoder_acc_session_mean['MOs'][predict_var][sel_stim],axis=1)
        ,'b.-',linewidth=2)

mos_line_balance_running=ax.plot(time_bins[1:],
                                  np.nanmean(decoder_acc_session_shuffle_mean['MOs'][predict_var][sel_stim],axis=1)
                                  ,'b^--',linewidth=1.5)

# ax.plot(time_bins[1:],np.nanmean(decoder_acc_session_mean['MRN'][predict_var][sel_stim],axis=1)
#         ,'r.-',linewidth=2)
# ax.plot(time_bins[1:],np.nanmean(decoder_acc_session_mean['CP'][predict_var][sel_stim],axis=1)
#         ,'g.-',linewidth=2)


ax.set_ylim([0.4,1.0])
ax.set_xlabel('time relative to stim onset (s)')
ax.set_ylabel('block decoding accuracy')
ax.set_title('decode: '+sel_stim)
# ax.legend([visp_line[0],mos_line[0]],['VISp','MOs'])
# ax.legend([visp_line[0],visp_line_balance_running[0],mos_line[0],mos_line_balance_running[0]],
#           ['VISp','VISp-balanced running','MOs','MOs-balanced running'])
ax.legend([visp_line[0],visp_line_balance_running[0],mos_line[0],mos_line_balance_running[0]],
          ['VISp','VISp-shuffle','MOs','MOs-shuffle'])

In [ ]:
predict_var='block_ids'
sel_stim='vis1'

fig,ax=plt.subplots(1,1)

ax.axvline(0,color='k',linestyle='--',linewidth=1)
ax.axhline(0.5,color='k',linestyle='--',linewidth=1)


visp_line=ax.errorbar(time_bins[1:],np.nanmean(decoder_acc_session_mean['VISp'][predict_var][sel_stim],axis=1),
                      yerr=np.nanstd(decoder_acc_session_mean['VISp'][predict_var][sel_stim],axis=1)/
                      np.sqrt(decoder_acc_session_mean['VISp'][predict_var][sel_stim].shape[1]),
                      fmt='k.-',linewidth=2, capsize=2)


visp_line_balance_running=ax.errorbar(time_bins[1:],
                                    np.nanmean(decoder_acc_session_shuffle_mean['VISp'][predict_var][sel_stim],axis=1),
                                    yerr=np.nanstd(decoder_acc_session_shuffle_mean['VISp'][predict_var][sel_stim],axis=1)/
                                    np.sqrt(decoder_acc_session_shuffle_mean['VISp'][predict_var][sel_stim].shape[1]),
                                    fmt='k^--',linewidth=1.5, capsize=2)


mos_line=ax.errorbar(time_bins[1:],np.nanmean(decoder_acc_session_mean['MOs'][predict_var][sel_stim],axis=1),
                     yerr=np.nanstd(decoder_acc_session_mean['MOs'][predict_var][sel_stim],axis=1)/
                     np.sqrt(decoder_acc_session_mean['MOs'][predict_var][sel_stim].shape[1]),
                     fmt='b.-',linewidth=2, capsize=2)


mos_line_balance_running=ax.errorbar(time_bins[1:],
                                    np.nanmean(decoder_acc_session_shuffle_mean['MOs'][predict_var][sel_stim],axis=1),
                                    yerr=np.nanstd(decoder_acc_session_shuffle_mean['MOs'][predict_var][sel_stim],axis=1)/
                                    np.sqrt(decoder_acc_session_shuffle_mean['MOs'][predict_var][sel_stim].shape[1]),
                                    fmt='b^--',linewidth=1.5, capsize=2)



ax.set_ylim([0.4,1.0])
ax.set_xlabel('time relative to stim onset (s)')
ax.set_ylabel('block decoding accuracy')
ax.set_title('decode: '+sel_stim)
# ax.legend([visp_line[0],mos_line[0]],['VISp','MOs'])
# ax.legend([visp_line[0],visp_line_balance_running[0],mos_line[0],mos_line_balance_running[0]],
#           ['VISp','VISp-balanced running','MOs','MOs-balanced running'])
ax.legend([visp_line[0],visp_line_balance_running[0],mos_line[0],mos_line_balance_running[0]],
          ['VISp','VISp-shuffle','MOs','MOs-shuffle'])

In [ ]:
all_areas

In [ ]:
#shaded error bars - MOs only

predict_var='block_ids'
sel_stim='vis1'

sel_area='VPM'

fig,ax=plt.subplots(1,1)

ax.axvline(0,color='k',linestyle='--',linewidth=1)
# ax.axhline(0.25,color='k',linestyle='--',linewidth=1)


y1=np.nanmean(decoder_acc_session_mean[sel_area][predict_var][sel_stim],axis=1)
err1=(np.nanstd(decoder_acc_session_mean[sel_area][predict_var][sel_stim],axis=1)/
      np.sqrt(np.sum(~np.isnan(decoder_acc_session_mean[sel_area][predict_var][sel_stim][0,:]))))
l1=ax.plot(time_bins[1:], y1,'k',linestyle='-')
plt.fill_between(time_bins[1:], y1-err1, y1+err1,
    alpha=0.2, edgecolor=None, facecolor='k')


y2=np.nanmean(decoder_acc_session_shuffle_mean[sel_area][predict_var][sel_stim],axis=1)
err2=(np.nanstd(decoder_acc_session_shuffle_mean[sel_area][predict_var][sel_stim],axis=1)/
      np.sqrt(np.sum(~np.isnan(decoder_acc_session_shuffle_mean[sel_area][predict_var][sel_stim][0,:]))))
l2=ax.plot(time_bins[1:], y2,'k',linestyle='--')
plt.fill_between(time_bins[1:], y2-err1, y2+err1,
    alpha=0.2, edgecolor=None, facecolor='k')



ax.set_ylim([0.44,1.0])
# ax.set_ylim([0.15,1.0])
ax.set_xlabel('time relative to stim onset (s)')
ax.set_ylabel('stimulus decoding accuracy')
ax.set_title('decode: '+sel_stim)
# ax.legend([visp_line[0],mos_line[0]],['VISp','MOs'])
# ax.legend([visp_line[0],visp_line_balance_running[0],mos_line[0],mos_line_balance_running[0]],
#           ['VISp','VISp-balanced running','MOs','MOs-balanced running'])
ax.legend([l1[0],l2[0]],[sel_area,sel_area+'-shuffle'])
fig.tight_layout()

In [ ]:
#multiple areas - sharded error bars

#shaded error bars - MOs only

predict_var='block_ids'
sel_stim='vis1'

fig,ax=plt.subplots(1,1)

ax.axvline(0,color='k',linestyle='--',linewidth=1)
# ax.axhline(0.25,color='k',linestyle='--',linewidth=1)


y1=np.nanmean(decoder_acc_session_mean['MOs'][predict_var][sel_stim],axis=1)
err1=(np.nanstd(decoder_acc_session_mean['MOs'][predict_var][sel_stim],axis=1)/
      np.sqrt(np.sum(~np.isnan(decoder_acc_session_mean['MOs'][predict_var][sel_stim][0,:]))))
l1=ax.plot(time_bins[1:], y1,'k',linestyle='-')
plt.fill_between(time_bins[1:], y1-err1, y1+err1,
    alpha=0.2, edgecolor=None, facecolor='k')


y2=np.nanmean(decoder_acc_session_shuffle_mean['MOs'][predict_var][sel_stim],axis=1)
err2=(np.nanstd(decoder_acc_session_shuffle_mean['MOs'][predict_var][sel_stim],axis=1)/
      np.sqrt(np.sum(~np.isnan(decoder_acc_session_shuffle_mean['MOs'][predict_var][sel_stim][0,:]))))
l2=ax.plot(time_bins[1:], y2,'k',linestyle='--')
plt.fill_between(time_bins[1:], y2-err1, y2+err1,
    alpha=0.2, edgecolor=None, facecolor='k')


ax.set_ylim([0.44,1.0])
# ax.set_ylim([0.15,1.0])
ax.set_xlabel('time relative to stim onset (s)')
ax.set_ylabel('stimulus decoding accuracy')
ax.set_title('decode: '+sel_stim)
# ax.legend([visp_line[0],mos_line[0]],['VISp','MOs'])
# ax.legend([visp_line[0],visp_line_balance_running[0],mos_line[0],mos_line_balance_running[0]],
#           ['VISp','VISp-balanced running','MOs','MOs-balanced running'])
ax.legend([l1[0],l2[0]],['MOs','MOs-shuffle'])
fig.tight_layout()



In [ ]:
# os.listdir(loadpath)
# decoder_acc_session_mean['VISp'][predict_var][sel_stim].shape[1]
np.sum(~np.isnan(decoder_acc_session_mean['MOs'][predict_var][sel_stim][0,:]))

In [ ]:
# with open(os.path.join(loadpath,decoder_results[0]),'rb') as handle:
#     X=pickle.load(handle)        
# X


In [ ]:
# X['metadata']